# Simplex tableau algorithm

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

import sys
from enum import Enum, auto

import numpy as np
from teaching_optimization.tableau import SimplexTableau, RowColumn


Consider the following linear optimization problem
$$
\max 4x_0 - 3x_1
$$
subject to
\begin{align*}
2x_0  +  x_1 &\leq  6,\\
x_0 - x_1    &\leq  2,\\
x_{0}  ,  x_{1}  &\geq  0. \\
\end{align*}

The objective of this exercise is to implement the simplex tableau algorithm to find the optimal solution.


First,  we write the problem in standard form:
$$
\min - 4x_0 +3x_1
$$
subject to
\begin{align*}
2x_0  +  x_1 + x_2 & =  6,\\
x_0 - x_1 + x_3    & =  2,\\
x_{0}  ,  x_{1}, x_2, x_3  &\geq  0.
\end{align*}

In [ ]:
standard_a = np.array([[2, 1, 1, 0], [1, -1, 0, 1]])
standard_b = np.array([6, 2])
standard_c = np.array([-4, 3, 0, 0])


As the right hand side is non negative, we choose the slack variables as basic variables, so that the basic
matrix is the identity matrix. The first tableau contains the data of the problem:

$$ \begin{array}{c|c} A & b \\ \hline c^T & 0 \end{array}$$

We merge A and b horizontally

In [ ]:
Ab = np.column_stack((standard_a, standard_b))


Then we add the last row

In [ ]:
initial_tableau = np.vstack(
    (Ab, np.append(standard_c, 0))
)


In [ ]:
the_tableau = SimplexTableau(initial_tableau)


First tableau

In [ ]:
print(the_tableau)


Details

In [ ]:
print(the_tableau.report())



Write a function that identifies a non-basic variable to enter the basis. Complete the ????

In [ ]:
def column_entering_basis(simplex_tableau: SimplexTableau) -> int | None:
    """
    Function that identifies a non-basic index to enter the basis, or detect optimality
    :param simplex simplex_tableau: object representing the tableau.
    :return: a non-basic index corresponding to a negative reduced cost, or None if optimal.
    """
    # the_tableau.tableau contains the numpy array with the entries of the tableau.
    # This can be done in one line using the following logic in numpy.
    #
    # - the_tableau.tableau[-1]: This selects the last row of the array.
    # - the_tableau.tableau[-1] < 0: This creates a boolean array where each element is True if the corresponding
    #   element in the last row of A is negative, and False otherwise.
    # - np.where(the_tableau.tableau[-1] < 0)[0]: np.where returns the indices of the elements that are True.
    #   The [0] extracts the first array of indices since np.where can potentially return a tuple
    #   of arrays if used on a multi-dimensional array.
    # - the second [0]: This selects the first index from the array of indices, which corresponds to
    #   the left-most negative value.
    # - if np.any(the_tableau.tableau[-1] < 0) else None: The if condition checks if there is any negative element
    #   in the last row. If not, it returns None.
    index = (
        np.where(simplex_tableau.tableau[-1] < 0)[0][0]
        if np.any(simplex_tableau.tableau[-1] < 0)
        else None
    )
    return index



We test the function. Expected result: Entering variable: 0

In [ ]:
the_entering_column = column_entering_basis(simplex_tableau=the_tableau)
print(f'Entering column: {the_entering_column}')


Verify the reduced cost for this variable. It must be negative.

In [ ]:
print(f'Reduced cost: {the_tableau.tableau[-1, the_entering_column]}')



Write a function that identifies a basic index to leave the basis.

In [ ]:
def row_leaving_basis(
    simplex_tableau: SimplexTableau,
    column_entering: int,
) -> int | None:
    """function that identifies a row corresponding to the basic variable leaving the basis, or identify an unbounded
        problem.

    :param simplex simplex_tableau: object representing the tableau.
    :param column_entering: non-basic index entering the basis
    :return: index of the variable leaving the basis, or None if unbounded
    """
    # First, identify the rows with positive entries.
    positive_row_indices = np.where(
        simplex_tableau.tableau[:, column_entering] > 0
    )[
        0
    ]

    # If there is no such entry, the problem is unbounded.
    if positive_row_indices.size == 0:
        return None

    # Calculate the vector of alphas
    vector_of_alphas = (
        simplex_tableau.tableau[positive_row_indices, -1]
        / simplex_tableau.tableau[positive_row_indices, column_entering]
    )

    # Find the index of the smallest alpha in vector_of_alphas
    min_alpha_index = np.argmin(vector_of_alphas)

    # Map the local index of the smallest alpha to the global index in the_tableau.
    row_index = int(positive_row_indices[min_alpha_index])

    return row_index



We test the function. Expected result: Exiting row: 1 (corresponding to variable $x_3$).

In [ ]:
the_exiting_row = row_leaving_basis(
    simplex_tableau=the_tableau,
    column_entering=the_entering_column,
)

print(f'Exiting row: {the_exiting_row}')



We define a list of possible interruptions of the algorithm

In [ ]:
class CauseInterruptionIterations(Enum):
    OPTIMAL = auto()
    UNBOUNDED = auto()
    INFEASIBLE = auto()

    def __str__(self) -> str:
        messages = {
            self.OPTIMAL: 'Optimal basis found.',
            self.UNBOUNDED: 'Optimization problem is unbounded.',
            self.INFEASIBLE: 'Optimization problem is infeasible.',
        }
        return messages[self]



Write a function that performs one iteration of the simplex algorithm.

In [ ]:
def simplex_iteration(
    simplex_tableau: SimplexTableau,
) -> tuple[SimplexTableau | None, CauseInterruptionIterations | None]:
    """
    Performs one iteration of the simplex algorithm.
    :param simplex_tableau: object representing the first tableau.
    :return: the new tableau, if successful. If None, a message explaining the reason.
    """
    entering_column = column_entering_basis(simplex_tableau=simplex_tableau)
    if entering_column is None:
        # Optimal solution found.
        return None, CauseInterruptionIterations.OPTIMAL

    exiting_row = row_leaving_basis(
        simplex_tableau=simplex_tableau,
        column_entering=entering_column,
    )
    if exiting_row is None:
        # Problem is unbounded.
        return None, CauseInterruptionIterations.UNBOUNDED

    pivot = RowColumn(row=exiting_row, column=entering_column)
    simplex_tableau.pivoting(pivot=pivot)
    return simplex_tableau, None



First iteration

In [ ]:
tableau_1, interruption = simplex_iteration(
    simplex_tableau=the_tableau,
)
if tableau_1 is None:
    print(interruption)


New tableau

In [ ]:
print(tableau_1)


Details

In [ ]:
print(tableau_1.report())


Second iteration

In [ ]:
tableau_2, interruption = simplex_iteration(simplex_tableau=tableau_1)
if tableau_2 is None:
    print(interruption)


New tableau

In [ ]:
print(tableau_2)


Details

In [ ]:
print(tableau_2.report())


Third iteration

In [ ]:
tableau_3, interruption = simplex_iteration(simplex_tableau=tableau_2)
if tableau_3 is None:
    print(interruption)